<a href="https://colab.research.google.com/github/rfr27/QA-Inicio/blob/main/haystack_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#asegurar que esta el CPU conectado  
%%bash

nvidia-smi

Thu Apr 27 23:06:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


# Inicializar DocumentStore

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [5]:
#iniciar el servidor y esperar hasta que este completamente instalado 
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch
#si usara docker seria con la función def launch_opensearch(sleep=15, delete_existing=False, local_port=9200)
 
 import time
time.sleep(30)

In [6]:
from haystack.utils import launch_es
launch_es()

In [7]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

#Preparamos los documentos


In [8]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/build_your_first_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",
    output_dir=doc_dir
)

INFO:haystack.utils.import_utils:Found data stored in 'data/build_your_first_question_answering_system'. Delete this first if you really want to fetch new data.


False

In [9]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

In [10]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

In [11]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/183 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/183 [00:00<?, ?docs/s]

{'documents': [<Document: {'content': '\n"\'\'\'Dark Wings, Dark Words\'\'\'" is the second episode of the third season of HBO\'s fantasy television series \'\'Game of Thrones\'\', and the 22nd episode of the series. Written by Vanessa Taylor, and directed by Daniel Minahan, it aired on April 7, 2013.\n\nThe title is an in-universe old saying about messenger ravens, referring to the fact that such urgently delivered messages are often bad news. In the episode, Robb Stark receives news of the death of Hoster Tully as well as the sack of Winterfell and the disappearances of Bran and Rickon.\n\n==Plot==\n===In King\'s Landing===\nMargaery and her grandmother Lady Olenna persuade Sansa to tell them the truth of King Joffrey\'s cruelty.\n\nAfter discussing his bride-to-be with Cersei, Joffrey invites Margaery to his chamber and questions her about her last husband, Renly Baratheon, and shows off his new crossbow.\n\nShae warns Tyrion that Lord Baelish has taken an interest in Sansa.\n\n===B

# Inicializar el Retriever

In [12]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


# Inicializar el Reader


In [13]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


# Crear el Retriever-Reader Pipeline


In [14]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

# Hacer una pregunta

In [20]:
prediction = querying_pipeline.run(
    query="who is hodor?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [21]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Wylis', 'type': 'extractive', 'score': 0.8126192688941956, 'context': '\nBran learns through visions of the past that, as a boy, Hodor was named Wylis and possessed normal abilities of speech. When the cave is overrun by W', 'offsets_in_document': [{'start': 87, 'end': 92}], 'offsets_in_context': [{'start': 73, 'end': 78}], 'document_ids': ['9023ff44cf063301cfd6cdea8885ed5'], 'meta': {'_split_id': 3, '_split_overlap': [{'range': [0, 134], 'doc_id': 'fe0fe10cc0fffa2f6c6817b8be0be484'}]}}>,
             <Answer {'answer': 'a simple-minded stable-boy at Winterfell', 'type': 'extractive', 'score': 0.6195346713066101, 'context': 'on.\n\n== Character description ==\n===Novels===\nHodor is a simple-minded stable-boy at Winterfell. He is popularly known as Hodor because that is the on', 'offsets_in_document': [{'start': 822, 'end': 862}], 'offsets_in_context': [{'start': 55, 'end': 95}], 'document_ids': ['a73cba99d9083df6db29aaf3d0c6b8f3'], 'meta': {'_split_id'

In [22]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

'Query: who is hodor?'
'Answers:'
[   {   'answer': 'Wylis',
        'context': '\n'
                   'Bran learns through visions of the past that, as a boy, '
                   'Hodor was named Wylis and possessed normal abilities of '
                   'speech. When the cave is overrun by W'},
    {   'answer': 'a simple-minded stable-boy at Winterfell',
        'context': 'on.\n'
                   '\n'
                   '== Character description ==\n'
                   '===Novels===\n'
                   'Hodor is a simple-minded stable-boy at Winterfell. He is '
                   'popularly known as Hodor because that is the on'},
    {   'answer': 'great-grandmother',
        'context': ' is employed to carry him in a sling on his back. Old Nan '
                   "(Hodor's great-grandmother) reveals to Bran that Hodor's "
                   'real name is Walder. When Winterfel'},
    {   'answer': 'Hodor is a huge, physically strong and intellectually '
                